In [6]:
import argparse
import sys
import numpy as np
from glob import glob

from KLT_filter.injections.rfi_shion_version import clean_persistent_rfi

import baseband_analysis.core.calibration as cal
import copy
from outriggers_vlbi_pipeline.known_calibrators import get_true_pulsar_pos
from ch_util import tools
import re
import baseband_analysis.core.bbdata as bbdata
from baseband_analysis import utilities
from baseband_analysis.analysis import beamform
import baseband_analysis.core.calibration as cal
from datetime import datetime
from outriggers_vlbi_pipeline.vlbi_pipeline_config import (
    VeryBasicBackend,
    kko_backend,
    gbo_backend,
    chime_backend,
    gbo_backend,
    valid_telescopes,
    current_version,
    kko_events_database,
    get_file_path
)
from baseband_analysis.core.bbdata import BBData
from glob import glob

from outriggers_vlbi_pipeline.query_database import get_event_data, update_event_status,get_full_filepath,find_files
from outriggers_vlbi_pipeline.multibeamform import get_calfiles
from outriggers_vlbi_pipeline.arc_commands import datatrail_pull_or_clear, datatrail_pull_cmd,datatrail_clear_cmd,baseband_exists,delete_baseband,vchmod,delete_multibeam,data_exists_at_minoc,datatrail_pull,datatrail_clear
from KLT_filter.injections.simulate_signal import inject_signal_into_baseband,generate_time_shifted_signal,inject_signal_into_baseband_2
import astropy.units as un

import astropy.coordinates as ac

chime = ac.EarthLocation.from_geocentric(
    x=-2059166.313 * un.m, y=-3621302.972 * un.m, z=4814304.113 * un.m
)
chime.info.name = "chime"

kko = ac.EarthLocation.from_geocentric(
    x = (-2111738.254-13.990) * un.m,
    y = (-3581458.222+4.666) * un.m, 
    z = (4821611.987-1.906) * un.m
)
kko.info.name = 'kko'



def generate_gaussian_signal(
    data,
    frame_start=100,
    frame_stop=1000):
    out_signal=np.zeros(data.ntime,dtype=data['baseband'].dtype)
    t_vals=np.arange(frame_start,frame_stop)
    center=(frame_stop+frame_start)//2
    width=(frame_start-frame_stop)//2
    N=len(t_vals)
    signal=(np.random.normal(0,1,N)+1j*np.random.normal(0,1,N))*np.exp(-((t_vals-center)/width)**2)
    out_signal[frame_start:frame_stop]+=signal
    return out_signal
 

def freq_id_from_filename(
    filename: str
) -> int:
    """Gets frequency id from filename of raw baseband data; in principle faster but perhaps less reliable than just extracting the header information from the .h5 data.
    Assumes that rawdata is written in the form baseband_<EVENT_ID>_FREQ_ID.h5

    Inputs: 
    ----------
    filename: str
        Name of raw baseband file
    Returns: 
    ----------
    frequency id corresponding corresponding to raw beamformed data
    """
    
    try:
        freq_id = re.split("_|.h5", filename)[-2]
        return int(freq_id)
    except:
        raise AssertionError(
            "raw data is not of the form baseband_<EVENT_ID>_FREQ_ID.h5."
        )

In [7]:
files= glob(f'/arc/projects/chime_frb/shiona_public/rfi_analysis/kko_rawdata/astro_307063854/.h5')

In [12]:
import pandas
signal_file=f'/arc/home/shiona/KLT_filter/sim_data/307063854_signal_simple.csv'
df=pandas.read_csv(signal_file)
signal_full=np.array(df['signal'][:].astype(complex))

In [19]:
ra=24.832135770543236
dec=60

In [20]:
s_t_n=65


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 s_t_ns=[65]*len(decs)                                                                        │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'decs' is not defined

In [21]:
glob('/arc/home/shiona/kko_interpolated_gains_scaled/*')

['/arc/home/shiona/kko_interpolated_gains_scaled/gain_20230204T190330.756782Z_CYG_A_interpolated.h5']

In [ ]:
data_clean = bbdata.BBData.from_acq_h5(clean_file)
ctime = data_clean["time0"]["ctime"][0]

noise=np.nanstd((np.abs(np.nansum(data_clean['baseband'][0,:],axis=0))**2)[frame_start:frame_stop],axis=-1) #shape ntime, std over ntime after summing inputs 
amplitude=np.sqrt(noise)*s_t_n/(np.sqrt(1000)*np.sqrt(len(new_files))) #approximately how many inputs there are in chime data, scale by n frequency channels 

telescope='kko'
inputs = tools.get_correlator_inputs(date, correlator=backend.correlator)
backend=kko_backend
reference_feed = tools.ArrayAntenna(
        id=-1, slot=-1, powered=True, flag=True, pos=[0, 0, 0], delay=0
) 
static_delays=False
cal_h5='/arc/home/shiona/kko_interpolated_gains_scaled/gain_20230204T190330.756782Z_CYG_A_interpolated.h5'

#telescope='chime'
#backend=chime_backend
#reference_feed=tools.CHIMEAntenna(id=-1, slot=-1, powered=True, flag=True, pos=[0, 0, 0])
#static_delays = backend.static_delays
#cal_h5 = get_calfiles(day=day,month=month,year=year,telescope=telescope, ctime=ctime)
#file='/arc/home/shiona/chime_inputs.pkl'
#import pandas as pd
#inputs = pd.read_pickle(file)

gains = cal.read_gains(cal_h5)
import os
output_file_dir=f'/arc/projects/chime_frb/shiona_public/rfi_analysis/processed_data/astro_{event_id}'
os.umask(0)
os.makedirs(
    output_file_dir, exist_ok=True, mode=0o777
)  # if output directory doesn't exist, create it

file='/arc/home/shiona/chime_inputs.pkl'
import pandas as pd
inputs = pd.read_pickle(file)

out_file=f'{output_file_dir}{event_id}_{telescope}_singlebeam_clean.h5'



datas=[]
date = datetime.utcfromtimestamp(ctime)


from simulate_signal import get_input_delays

# order feed positions (ordered to bbdata)
converted_data_inputs = data_clean.input.astype(
        [("chan_id", "<u2"), ("correlator_input", "U32")]
    )
reordered_inputs = tools.reorder_correlator_inputs(
            converted_data_inputs, inputs
        )

reordered_inputs.append(reference_feed)

if telescope=='chime':
    station=chime
    static_delays=True
if telescope=='kko':
    station=kko
    static_delays=False

# we ignore dispersive delay over the channel for now
antenna_delays,baseline_delay=get_input_delays(feeds=reordered_inputs,station=station,ctime=ctime,ra=ra,dec=dec,static_delays=static_delays)
total_delay=antenna_delays+baseline_delay
chunk_size=min(1,len(datafiles_all))

print(f"CHUNKS: {len(datafiles_all)//chunk_size}")
for i in range(len(datafiles_all)//chunk_size):
    print(f"{i*chunk_size}-{i*chunk_size+chunk_size} of {len(datafiles_all)} for telescope {telescope}!")
    files_to_process=datafiles_all[i*chunk_size:i*chunk_size+chunk_size]

    data = bbdata.BBData.from_acq_h5(files_to_process)

    # remove cable delays etc before injecting signal into baseband data 
    cal.apply_calibration(data, gains, inputs=inputs)

    if telescope!='chime':
        inject_signal_into_baseband(data,np.array(signal)*amplitude,delays=total_delay,s_t_n=s_t_n2,tec=TEC,inputs=inputs,frame_start=frame_start,frame_stop=frame_stop)
        #inject_signal_into_baseband_2(data,np.array(signal),delays=total_delay,s_t_n=s_t_n2,tec=TEC,inputs=inputs,frame_start=frame_start,frame_stop=frame_stop,power=power,
        #digital_gain_file=digital_gain_file,gain_file=cal_h5)
    else:
        inject_signal_into_baseband(data,np.array(signal)*amplitude,delays=total_delay,s_t_n=s_t_n2,tec=0,inputs=inputs,frame_start=frame_start,frame_stop=frame_stop)
        #inject_signal_into_baseband_2(data,np.array(signal),delays=total_delay,s_t_n=s_t_n2,tec=0,inputs=inputs,frame_start=frame_start,frame_stop=frame_stop,power=power,
        #digital_gain_file=digital_gain_file,gain_file=cal_h5)
    if POL:
        clean_persistent_rfi_pol(
            data=data, ra=np.array([ra]), dec=np.array([dec]), ps_gain_file=cal_h5,inputs=inputs,
            reference_feed=reference_feed,static_delays=static_delays,clean=clean,
            obs=backend.obs,
            Ex_Ex=Ex_Ex,
            Ey_Ey=Ey_Ey,
            Ex_Ey=Ex_Ey,
            Ey_Ex=Ey_Ex,
        frame_start=frame_start,frame_stop=frame_stop) # fringestop_delays=antenna_delays,

    else:
        clean_persistent_rfi(
            data=data, ra=np.array([ra]), dec=np.array([dec]), ps_gain_file=cal_h5,inputs=inputs,
            reference_feed=reference_feed,static_delays=static_delays,clean=clean,
            obs=backend.obs,source_names=np.array(['placeholder_name']),
        frame_start=frame_start,frame_stop=frame_stop)


    if telescope!='chime':
        #fringestop data to chime
        print("fringe stopping")
        fringedstopped_tiedbeam=generate_time_shifted_signal(
            copy.deepcopy(data['tiedbeam_baseband'][:,:,:]),
            -baseline_delay, #microseconds
            data.freq, #Mhz, either float or array of floats
            beamformed=True,
            )
        data['tiedbeam_baseband'][:,:,:]=fringedstopped_tiedbeam

    datas.append(data)



beamformed_rfi_cleaned = bbdata.concatenate(datas)
print(out_file)
beamformed_rfi_cleaned.save(out_file)
beamformed_rfi_cleaned.close()
#datatrail_pull_or_clear(event_id, telescope=telescope, cmd_str='CLEARED')
